# Tool for Sending Logged-in Users a Reminder to Log Out for Maintenance

In [ ]:
%%capture
# requirement for emails
!{sys.executable} -m pip install secure-smtplib 

# requirements for database access
!git clone https://github.com/ITISFoundation/osparc-simcore.git
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install psycopg2-binary
!{sys.executable} -m pip install pydantic aioredis
!{sys.executable} -m pip install pytz

import os
import datetime
import pytz
from pytz import timezone
zurichTZ = timezone('Europe/Zurich')
from dateutil.relativedelta import relativedelta

In [ ]:
# A list of user IDs you would like to send emails to - these can be found from redis commander
excluded_users = ["kuster@itis.swiss", "@itis.testing"] # can be only aprt of a mail address 

# Provide the time of maintenance
# datetime(year, month, day, hour, minute, second)
maintenance_time = datetime.datetime(2022, 8, 15, 17, 15, 0, tzinfo=zurichTZ)

productString = "o²S²PARC / S4L-Web / TI-Planning"
deploymentString = os.environ.get('DEPLOYMENT_FQDNS')

In [ ]:

# Compute timing inforation
utc = pytz.timezone('UTC')
now = utc.localize(datetime.datetime.utcnow())
attrs = ['years', 'months', 'days', 'hours', 'minutes', 'seconds']
human_readable = lambda delta: ['%d %s' % (getattr(delta, attr), attr if getattr(delta, attr) > 1 else attr[:-1]) for attr in attrs if getattr(delta, attr)]
assert(maintenance_time > now)
timeDiff = human_readable(relativedelta(seconds=(int(maintenance_time.strftime('%s')) - int(now.strftime('%s')))))
maintenanceInStr = " ".join(timeDiff[:2])
print("Maintenance in: ",maintenanceInStr)
maintenance_timeStr = maintenance_time.strftime('%H:%M')
print("Maintenance at: ",maintenance_timeStr)
maintenance_dateStr = maintenance_time.strftime('%A %d. %B %Y')
print("Maintenance on: ",maintenance_dateStr)

In [ ]:
# REDIS
# from settings_library.redis import RedisSettings
from pydantic.networks import RedisDsn
import aioredis

REDIS_HOST = os.environ.get('REDIS_HOST')
REDIS_PORT = os.environ.get('REDIS_PORT')
REDIS_DB = "0"
REDIS_USER = None
REDIS_PASSWORD = None

user_ids_projectOpen = []

settings = RedisDsn.build(
            scheme="redis",
            user = REDIS_USER or None,
            password = REDIS_PASSWORD if REDIS_PASSWORD else None,
            host = REDIS_HOST,
            port = REDIS_PORT,
            db = REDIS_DB
        )

# settings = RedisSettings() # This captures automatically env vars

client = aioredis.from_url(
                    settings, encoding="utf-8", decode_responses=True
                )
await client.ping()
listOfResourceKeys = [ hash_key async for hash_key in client.scan_iter(match=f"*") if ':resources' in hash_key]
#listOfResourceValues = [await client.hgetall(hash_key) for hash_key in listOfResourceKeys]
for i in listOfResourceKeys:
    currentValues = await client.hgetall(i)
    if 'socket_id' in currentValues and 'project_id' in currentValues: # user has a proejct open
        currentUserID = int(i.split(':')[0].split('user_id=')[1])
        user_ids_projectOpen.append(currentUserID)

## Access the Database to Find Names and Emails

In [ ]:
import sqlalchemy as db
import psycopg2
import os
import sys
import json
import importlib
from pathlib import Path
#########################################
#######################
PG_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
PG_ENDPOINT=os.environ.get('POSTGRES_ENDPOINT')
PG_DB=os.environ.get('POSTGRES_DB')
PG_USER=os.environ.get('POSTGRES_USER')
pgEngineURL= "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        user=PG_USER,
        password=PG_PASSWORD,
        database=PG_DB,
        host=PG_ENDPOINT.split(":")[0],
        port=int(PG_ENDPOINT.split(":")[1]),
    )
engine = db.create_engine(pgEngineURL)
connection = engine.connect()
metadata = db.MetaData()
user_table = db.Table('users', metadata, autoload=True, autoload_with=engine)

## Find These Users

In [ ]:
name_list = []
email_list = []

for user in user_ids_projectOpen:
    query = db.select([user_table]).where(user_table.c.id==user)
    result_proxy = connection.execute(query)
    result_set = result_proxy.fetchall()
    if result_set:
        user_data = dict(result_set[0])
        userIsExcluded = False
        for excluded_mail in excluded_users:
            if excluded_mail in user_data["email"]:
                userIsExcluded = True
                break
        if not userIsExcluded:
            name_list.append(user_data["name"])
            email_list.append(user_data["email"])
    else:
        print("User ID:" + str(user) + " not found in database")

## Log into Mail Server to Send Emails

In [ ]:
MACHINE_FQDN = os.environ.get('MACHINE_FQDN')
SMTP_HOST = os.environ.get('SMTP_HOST')
SMTP_PORT = os.environ.get('SMTP_PORT')
SMTP_USERNAME = os.environ.get('SMTP_USERNAME')
SMTP_PASSWORD =  os.environ.get('SMTP_PASSWORD')


## Send a Nice Email to Everyone Logged In

In [ ]:
## =======  For debugging =======

# name_list = ["Friend", "Frenemy"]
# email_list = ["something@itis.swiss", "somethingelse@gmail.com"]

In [ ]:
print("You are about to send emails to: ")
print('\n'.join(email_list))
print("\nPlease be sure before continuing...")

In [ ]:
import smtplib, ssl

context = ssl.create_default_context()
with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as server:
    server.ehlo()  # Can be omitted
    server.starttls(context=context)
    server.ehlo()  # Can be omitted
    server.login(SMTP_USERNAME, SMTP_PASSWORD)
    for name, email in zip(name_list, email_list):
        message = """Subject: """ + productString + """ Maintenance Warning

\nDear user """ + name + """,\n
You are currently logged in with an account on: """ + str(deploymentString).replace(',',' /') +""".
Please note that we will be doing a routine maintenance on the platform in """\
+ str(maintenanceInStr) + """. To keep your work safe, we recommend
that you save your studies and log out of  """ + str(deploymentString) +""". Apologies for
any inconvenience.

Cheers,
your friendly  """ + str(deploymentString) +""" team"""

        server.sendmail(SMTP_USERNAME, email, message.encode('utf-8'))